In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

## Package imports and settings 

In [2]:
import pandas as pd
import seaborn as sns
import datetime as dt
import cpi
import requests
import re
from unidecode import unidecode

/var/folders/wl/g3zsz07j19b8ryhr2_wvq_c00000gn/T/ipykernel_28727/733519477.py:4: StaleDataWarning: CPI data is out of date. To accurately inflate to today's dollars, you must run `cpi.update()`.
  import cpi


In [3]:
import wbgapi as wb
import wbdata
import pandas as pd
from countrycode import countrycode
import numpy as np
from currency_converter import CurrencyConverter

In [4]:
# Right after importing seaborn (could also use 'whitegrid')
sns.set_theme(style='whitegrid', context='talk')

In [5]:
# surpress scientific notation
pd.options.display.float_format = '{:.1f}'.format

# Importing Data

## Lobster Export Data

### Which codes do relate to lobsters?

From : https://www.ic.gc.ca/app/scr/tdst/tdo/crtr.html?grouped=INDIVIDUAL&searchType=KS_CS&naArea=9999&countryList=ALL&toFromCountry=CDN&reportType=TE&timePeriod=5%7CComplete+Years&currency=CDN&productType=HS6&hSelectedCodes=%7C10600%7C30611%7C30612%7C30615%7C30621%7C30622%7C30625%7C30631%7C30632%7C30634%7C30691%7C30692%7C30694%7C160530&runReport=true

Searched for all codes containing word "lobster". Extracted all codes within HS 1605 and HS 0306.

In [6]:
# jlab
path_1605 = 'full_data/1605_88Jan_to_23Sep_ATP_PV_Monthly.csv'
df_1605_full = pd.read_csv(path_1605, engine = 'python', skipfooter = 3,index_col=False, header = 1)
df_1605_full.head()

,Period,Commodity,Province,Country,State,Value ($),Quantity,Unit of measure
0,2013-04-01,"1605.21.00 - Shrimps and prawns, prepared or p...",Newfoundland and Labrador,United States,California,195111,29802,Weight in kilograms
1,2013-12-01,"1605.21.00 - Shrimps and prawns, prepared or p...",Newfoundland and Labrador,United States,California,285427,35966,Weight in kilograms
2,2017-10-01,"1605.59.00 - Molluscs, prepared or preserved, nes",Newfoundland and Labrador,United States,California,185609,22500,Weight in kilograms
3,2014-08-01,"1605.10.91 - Crab, snow (Queen), prepared or p...",Newfoundland and Labrador,United States,Alabama,191769,14969,Weight in kilograms
4,2013-01-01,"1605.21.00 - Shrimps and prawns, prepared or p...",Newfoundland and Labrador,United States,California,170470,16053,Weight in kilograms


In [7]:
path_0306 = 'full_data/0306-Crust-Jan88-to-Sep23-prov.csv'
df_0306_full = pd.read_csv(path_0306, engine = 'python', skipfooter = 3,index_col=False, header = 1)
df_0306_full.head()

,Period,Commodity,Province,Country,State,Value ($),Quantity,Unit of measure
0,2014-06-01,"0306.16.10 - Cold-water shrimps and prawns, fr...",Newfoundland and Labrador,Greenland,NaN,220915,66528,Weight in kilograms
1,2023-04-01,"0306.14.10 - Crabs, snow (Queen), frozen",Newfoundland and Labrador,United States,California,1111067,61916,Weight in kilograms
2,2015-01-01,"0306.16.10 - Cold-water shrimps and prawns, fr...",Newfoundland and Labrador,Greenland,NaN,753558,174576,Weight in kilograms
3,2015-06-01,"0306.16.10 - Cold-water shrimps and prawns, fr...",Newfoundland and Labrador,Greenland,NaN,883171,199392,Weight in kilograms
4,2015-07-01,"0306.17.10 - Shrimps and prawns, frozen, in sh...",Newfoundland and Labrador,Saint Pierre and Miquelon,NaN,704,45,Weight in kilograms


### Merge 1605 and 0306

In [8]:
# concat function
def concat_dfs(df_list):
    df = pd.concat(df_list, axis = 0, ignore_index = True)
    return df

In [9]:
full_data = concat_dfs([df_1605_full,df_0306_full])


## Inflation

In [10]:
# from fred, index 2015, 100
inflation_data = pd.read_csv('data/CPI-data-fred.csv')
inflation_data = inflation_data.rename(columns = {'DATE': 'Period', 'CPALCY01CAM661N':'cpi_val'})
inflation_data['Period'] = pd.to_datetime(inflation_data['Period'])

In [11]:
inflation_data

,Period,cpi_val
0,1988-01-01,56.9
1,1988-02-01,57.3
2,1988-03-01,57.5
3,1988-04-01,57.6
4,1988-05-01,58.0
...,...,...
424,2023-05-01,124.5
425,2023-06-01,125.1
426,2023-07-01,125.2
427,2023-08-01,126.0


# 1. Cleaning
---
---

## Initial Cleaning Steps

In [30]:
# perform cleaning set on combined data set
def cleaning_steps(df_name):
    # make lower case
    df_name['Commodity'] = df_name['Commodity'].str.lower()
    # keep only commodities that mention 'lobster'
    df_name = df_name[df_name['Commodity'].str.contains('lobster')].reset_index(drop=True)
    # drop extra columns
    df_name = df_name.drop(columns = ['Unit of measure'])
    # rename columns
    new_col_names = {'Value ($)': 'Value', 'Quantity': 'Quantity_kg'}
    df_name = df_name.rename(columns=new_col_names)
    # update period to datetime object
    df_name['Period'] = pd.to_datetime(df_name['Period'])
    # remove terminated from country name 
    df_name[["Country", 'Terminated']]= df_name["Country"].str.split(r'(',expand=True)
    # filter out non homarus americanus commodities:
    homarus_comm = ["0306.12",
"0306.22",
"0306.32",
"0306.92",
"1605.30"]
    df_name = df_name[df_name['Commodity'].str[:7].isin(homarus_comm)]
    return df_name



In [31]:
data_clean = cleaning_steps(full_data)
data_clean

,Period,Commodity,Province,Country,State,Value,Quantity_kg,Terminated
0,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",Newfoundland and Labrador,United States,Massachusetts,2454203,42059,None
1,2018-06-01,"1605.30.10 - lobster, prepared or preserved, f...",Newfoundland and Labrador,United States,Massachusetts,2199419,48158,None
2,2014-06-01,"1605.30.11 - lobster meat, frozen, cooked by s...",Newfoundland and Labrador,United States,Massachusetts,1299983,38486,None
3,2014-10-01,"1605.30.11 - lobster meat, frozen, cooked by s...",Newfoundland and Labrador,United States,Massachusetts,151542,3475,None
4,2014-11-01,"1605.30.11 - lobster meat, frozen, cooked by s...",Newfoundland and Labrador,United States,Massachusetts,634072,22628,None
...,...,...,...,...,...,...,...,...
57380,2006-02-01,"0306.22.10 - lobsters, (homarus spp), live (te...",Northwest Territories,Hong Kong,NaN,6000,273,None
57381,2007-02-01,"0306.22.10 - lobsters, (homarus spp), live (te...",Northwest Territories,Japan,NaN,7200,408,None
57382,2007-03-01,"0306.22.10 - lobsters, (homarus spp), live (te...",Northwest Territories,"Korea, South",NaN,10080,572,None
57383,2006-02-01,"0306.22.10 - lobsters, (homarus spp), live (te...",Nunavut,Czechia,NaN,3000,136,None


In [32]:
data_clean.loc[data_clean['Commodity'].str.contains("terminated"), 'Commodity'].unique()

array(['1605.30.11 - lobster meat, frozen, cooked by steaming or boiling in water (terminated 2016-12)',
       '1605.30.99 - lobster, prepared or preserved, o/t in airtight containers (terminated 2016-12)',
       '1605.30.19 - lobster meat, o/t frozen, cooked by steaming or boiling in water (terminated 2016-12)',
       '1605.30.91 - lobster, prepared or preserved, in airtight containers (terminated 2016-12)',
       '1605.30.11 - lobster meat, frozen, simply boiled in water (terminated 2016-12)',
       '1605.30.19 - lobster meat, not frozen, simply boiled in water (terminated 2016-12)',
       '1605.30.99 - lobster, prepared or preserved, not in airtight containers (terminated 2016-12)',
       '1605.30.12 - lobster meat, not frozen, simply boiled in water (terminated 1989-12)',
       '1605.30.92 - lobster, prepared or preserved, not in airtight containers (terminated 1989-12)',
       '0306.22.10 - lobsters, live (terminated 2016-12)',
       '0306.22.90 - lobsters, not frozen, o

In [34]:

def inflation_adjustment(df_name, cpi_df):
    ''' adjust monetary features into today's dolars, index 2015 = 100'''
    df_name = df_name.merge(cpi_df, how='inner', on='Period')
    today_cpi = cpi_df.loc[cpi_df['Period'] == cpi_df['Period'].max(), 'cpi_val'].item()
    # create new column to adjust to today's dollars
    # return df_name
    df_name['Value-adjusted'] = (df_name['Value'] / df_name['cpi_val']) * today_cpi
    return df_name
    
    

In [35]:
data_clean = inflation_adjustment(data_clean, inflation_data)
data_clean


,Period,Commodity,Province,Country,State,Value,Quantity_kg,Terminated,cpi_val,Value-adjusted
0,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",Newfoundland and Labrador,United States,Massachusetts,2454203,42059,None,102.2,3023516.5
1,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",New Brunswick,United States,Massachusetts,20137016,370393,None,102.2,24808298.7
2,2017-05-01,"1605.30.90 - lobster, prepared or preserved, nes",Prince Edward Island,United States,California,31113,1633,None,102.2,38330.4
3,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",Quebec,United States,California,98603,2347,None,102.2,121476.4
4,2017-05-01,"1605.30.90 - lobster, prepared or preserved, nes",Prince Edward Island,United States,New Hampshire,310914,15676,None,102.2,383038.3
...,...,...,...,...,...,...,...,...,...,...
56885,1989-02-01,"0306.22.10 - lobsters, (homarus spp), live (te...",British Columbia,Hong Kong,NaN,81553,4538,None,59.7,172082.3
56886,1989-02-01,"0306.12.10 - lobsters in brine, (homarus spp),...",Prince Edward Island,Sweden,NaN,107235,8340,None,59.7,226273.0
56887,1989-02-01,"0306.22.90 - lobsters,(homarus spp), not froze...",Nova Scotia,United States,Massachusetts,16048,1316,None,59.7,33862.4
56888,1989-02-01,"0306.22.10 - lobsters, (homarus spp), live (te...",Nova Scotia,United States,Unknown states,124924,8381,None,59.7,263598.0


In [36]:
data_clean.to_csv('data/data_clean.csv', index=False)

### make country list csv

In [37]:
export_country_list = pd.Series(data_clean['Country'].unique())
export_country_list = export_country_list.sort_values(ignore_index= True)
export_country_list[10:30]

10                             Barbados
11                              Belarus
12                              Belgium
13                               Belize
14                              Bermuda
15                              Bolivia
16     Bonaire, Sint Eustatius and Saba
17                               Brazil
18                             Bulgaria
19                         Burkina Faso
20                           Cabo Verde
21                             Cambodia
22                       Cayman Islands
23             Central African Republic
24                                Chile
25                                China
26                     Christmas Island
27                               Cocos 
28                             Colombia
29    Congo, Democratic Republic of the
dtype: object

In [38]:
export_country_list.to_csv('data/export_countries.csv')

# GDP Data

## Get GDP Data for each year and country GDP Using WBAPI

In [39]:
# list of countries that need GDP data for 
countries = pd.read_csv('data/export_countries.csv', index_col=0, header = 1, names = ['Country'])

In [40]:
countries

,Country
1,Albania
2,Antigua and Barbuda
3,Argentina
4,Armenia
5,Aruba
...,...
155,Uzbekistan
156,Venezuela
157,Viet Nam
158,West Germany


### WBAPI regions 
(not same as continent.)

In [41]:
economy_data = wb.economy.DataFrame()
economy_data

,name,aggregate,longitude,latitude,region,adminregion,lendingType,incomeLevel,capitalCity
id,,,,,,,,,
ABW,Aruba,False,-70.0,12.5,LCN,,LNX,HIC,Oranjestad
AFE,Africa Eastern and Southern,True,NaN,NaN,,,,,
AFG,Afghanistan,False,69.2,34.5,SAS,SAS,IDX,LIC,Kabul
AFW,Africa Western and Central,True,NaN,NaN,,,,,
AGO,Angola,False,13.2,-8.8,SSF,SSA,IBD,LMC,Luanda
...,...,...,...,...,...,...,...,...,...
XKX,Kosovo,False,20.9,42.6,ECS,ECA,IDX,UMC,Pristina
YEM,"Yemen, Rep.",False,44.2,15.4,MEA,MNA,IDX,LIC,Sana'a
ZAF,South Africa,False,28.2,-25.7,SSF,SSA,IBD,UMC,Pretoria


In [42]:
economy_data = economy_data.reset_index(drop=False)
economy_data = economy_data[['id', 'name', 'region']]
economy_data

,id,name,region
0,ABW,Aruba,LCN
1,AFE,Africa Eastern and Southern,
2,AFG,Afghanistan,SAS
3,AFW,Africa Western and Central,
4,AGO,Angola,SSF
...,...,...,...
261,XKX,Kosovo,ECS
262,YEM,"Yemen, Rep.",MEA
263,ZAF,South Africa,SSF
264,ZMB,Zambia,SSF


In [43]:
gdp_indicator = 'NY.GDP.MKTP.CD'

In [44]:
country_names = countries.iloc[:, 0].to_list()
country_names[:20]

['Albania',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Bahamas',
 'Bahrain',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Bermuda',
 'Bolivia',
 'Bonaire, Sint Eustatius and Saba',
 'Brazil',
 'Bulgaria',
 'Burkina Faso',
 'Cabo Verde']

In [45]:
# using world bank API, match country names. WBAPI will match on several types of country names that reuquire GDP data For
data = wb.economy.coder(country_names)
# explore data 
# print(data.items())
# print(dir(data))
# print(data.keys())

In [46]:
# store country codes in a list - all country codes in WBAPI
country_codes = list(data.values())
# country_codes 

In [47]:
# using country codes, get annual GDP by year for the relevant years in data 
gdp_data = wb.data.DataFrame(gdp_indicator, country_codes, time=range(1988, 2023)).reset_index()

##### Reformat GDP Data to Country, Year

In [48]:
gdp_data = pd.melt(gdp_data, id_vars = ['economy'], value_vars =list(gdp_data.columns[1:]))


In [49]:
# rename columns to make merge easier
gdp_data = gdp_data.rename(columns={'economy': 'country_code', 'variable': 'year', 'value': 'GDP-USD'})

# reformat year 
gdp_data['year'] = gdp_data['year'].str.lstrip('YR').astype('int')
gdp_data

,country_code,year,GDP-USD
0,ABW,1988,596648044.7
1,ALB,1988,2051236250.0
2,ARE,1988,36275674203.2
3,ARG,1988,126928000000.0
4,ARM,1988,NaN
...,...,...,...
4825,USA,2022,25439700000000.0
4826,UZB,2022,80391853887.4
4827,VEN,2022,NaN
4828,VNM,2022,408802378904.8


In [50]:
# convert GDP to CAD? 

# need gdp per capita too? 

### Mapping Countries - code, continent

In [51]:
# get continent for each country for visuals. 
continent_list = []
for country_code in country_codes:
    if country_code is None:
        continent_list.append(np.nan)
    else:
        continent_value = countrycode(country_code, origin = "iso3c", destination = "continent")
        continent_list.append(continent_value)

In [52]:
# add contient to countries df
countries['continent'] = continent_list
countries.tail(40)

,Country,continent
120,Russian Federation,Europe
121,"Saint Helena, Ascension and Tristan da Cunha",NaN
122,Saint Pierre and Miquelon,NaN
123,Saudi Arabia,Asia
124,Senegal,Africa
125,Serbia,Europe
126,Seychelles,Africa
127,Sierra Leone,Africa
128,Singapore,Asia
129,Slovakia,Europe


In [53]:
# add country code to contries df for easier mapping
countries['country_code'] = country_codes
countries[:20]

,Country,continent,country_code
1,Albania,Europe,ALB
2,Antigua and Barbuda,Americas,ATG
3,Argentina,Americas,ARG
4,Armenia,Asia,ARM
5,Aruba,Americas,ABW
6,Australia,Oceania,AUS
7,Austria,Europe,AUT
8,Bahamas,Americas,BHS
9,Bahrain,Asia,BHR
10,Barbados,Americas,BRB


In [54]:
# Export to csv for use later where needed
countries.to_csv('data/country_code_mapping.csv', index = False)

In [55]:
gdp_data

,country_code,year,GDP-USD
0,ABW,1988,596648044.7
1,ALB,1988,2051236250.0
2,ARE,1988,36275674203.2
3,ARG,1988,126928000000.0
4,ARM,1988,NaN
...,...,...,...
4825,USA,2022,25439700000000.0
4826,UZB,2022,80391853887.4
4827,VEN,2022,NaN
4828,VNM,2022,408802378904.8


In [56]:
gdp_data.to_csv('data/gdp-data-clean.csv')

### QA - Countries without GDP

In [57]:
len(data_clean['Country'].unique())

160

In [58]:
# countries need gdp for 
countries

,Country,continent,country_code
1,Albania,Europe,ALB
2,Antigua and Barbuda,Americas,ATG
3,Argentina,Americas,ARG
4,Armenia,Asia,ARM
5,Aruba,Americas,ABW
...,...,...,...
155,Uzbekistan,Asia,UZB
156,Venezuela,Americas,VEN
157,Viet Nam,Asia,VNM
158,West Germany,Europe,DEU


In [59]:
## Cross reference - drop countries if not significant 

no_gdp_countries = countries[countries['country_code'].isna()].reset_index(drop=True) # countries need gdp for 

In [60]:
no_gdp_countries

,Country,continent,country_code
0,"Bonaire, Sint Eustatius and Saba",NaN,None
1,Christmas Island,NaN,None
2,Cocos,NaN,None
3,Czechoslovakia,NaN,None
4,Former Union of Soviet Socialist Republics,NaN,None
5,French Southern Antarctic Territories,NaN,None
6,French Southern Territories,NaN,None
7,Guadeloupe,NaN,None
8,Heard Island and McDonald Islands,NaN,None
9,Martinique,NaN,None


In [61]:
no_gdp_countries2 = list(data_clean.loc[data_clean['Country'].isin(no_gdp_countries['Country']), 'Country'].unique())

In [62]:
no_gdp_countries2

['Christmas Island',
 'Saint Pierre and Miquelon',
 'Bonaire, Sint Eustatius and Saba',
 'Guadeloupe',
 'Former Union of Soviet Socialist Republics ',
 'Czechoslovakia ',
 'Saint Helena, Ascension and Tristan da Cunha',
 'Martinique',
 'French Southern Antarctic Territories',
 'French Southern Territories',
 'Tokelau',
 'Union of Soviet Socialist Republics ',
 'Yugoslavia ',
 'Heard Island and McDonald Islands',
 'Cocos ',
 'Norfolk Island']

In [63]:
years_list =  list(set(data_clean['Period'].dt.year.unique()))
# years_list

In [64]:
# sum of total exports by year for no gdp countries
def annual_value(df_name):
    result = pd.DataFrame()
    for country in no_gdp_countries['Country']:
        subtotal = df_name.copy()
        subtotal = subtotal.loc[subtotal['Country'].str.contains(country),:]
        subtotal['Year'] = subtotal['Period'].dt.year
        subtotal = pd.DataFrame(subtotal[['Country', 'Value-adjusted', 'Year']].groupby(['Country', 'Year']).sum())
        # print(subtotal)
        result = pd.concat([result, subtotal])
    result = result.reset_index()
    return result


In [65]:
result = annual_value(data_clean)

In [66]:
# what percent of total export value is no gdp countries?
# total_gdp_year 
total_ann_gdp = pd.DataFrame(data_clean['Value-adjusted'].groupby(data_clean['Period'].dt.year).sum()).reset_index().rename(columns={'Period': 'Year'})
total_ann_gdp.tail(10)

,Year,Value-adjusted
26,2014,1935712027.2
27,2015,2557895020.7
28,2016,2682042346.4
29,2017,2619642778.9
30,2018,2697240718.8
31,2019,3069326289.1
32,2020,2457183232.1
33,2021,3683366032.4
34,2022,2932282826.2
35,2023,2105291217.4


In [67]:
# what percent of total export value is no gdp countries by year?

# join on year - all values in result, matched on year
percent_gdp = pd.merge(result, total_ann_gdp, on = 'Year', suffixes=('', '_all_countries_year'))
percent_gdp

,Country,Year,Value-adjusted,Value-adjusted_all_countries_year
0,"Bonaire, Sint Eustatius and Saba",2018,40023.6,2697240718.8
1,Christmas Island,2018,642575.6,2697240718.8
2,"Bonaire, Sint Eustatius and Saba",2019,4609.1,3069326289.1
3,Christmas Island,2019,67571.4,3069326289.1
4,Christmas Island,2015,36470.9,2557895020.7
...,...,...,...,...
56,Saint Pierre and Miquelon,2011,2728.3,1366761075.5
57,Saint Pierre and Miquelon,2012,4027.7,1418669895.1
58,Saint Pierre and Miquelon,2013,2444.9,1486531211.3
59,Saint Pierre and Miquelon,2016,1288.3,2682042346.4


In [68]:
percent_gdp['Percent'] = percent_gdp['Value-adjusted'] / percent_gdp['Value-adjusted_all_countries_year']*100
percent_gdp.sort_values(by= 'Percent', ascending = False).head(10)

,Country,Year,Value-adjusted,Value-adjusted_all_countries_year,Percent
44,Saint Pierre and Miquelon,1996,526151.1,981313888.2,0.1
16,Saint Pierre and Miquelon,1993,280049.7,648616317.3,0.0
21,Saint Pierre and Miquelon,1989,217746.2,593837153.0,0.0
41,Norfolk Island,2001,490056.7,1528119870.9,0.0
27,Saint Pierre and Miquelon,1988,226753.7,737105498.9,0.0
38,Saint Pierre and Miquelon,1995,306097.8,998437927.2,0.0
35,Saint Pierre and Miquelon,1994,272077.1,897852808.0,0.0
31,Saint Pierre and Miquelon,1991,188542.8,630551937.6,0.0
43,Saint Pierre and Miquelon,1990,159146.8,613297117.6,0.0
11,Saint Pierre and Miquelon,1992,177861.0,701159259.2,0.0


In [69]:
no_gdp_sum = result['Value-adjusted'].sum()

In [70]:
gdp_sum = data_clean['Value-adjusted'].sum()

In [71]:
# ho much of total exports are in countries that dont have gdp data? 

(no_gdp_sum / gdp_sum) * 100

# less than 0.01% --> 99.99% of total export value have gdp data

0.008499649032854795

#### Decision:

Decision... drop records for countries without GDP data. 
Since counties are likely territories of other gdp states. ?

#### Replace Country Code with Country Name

In [72]:
countries

,Country,continent,country_code
1,Albania,Europe,ALB
2,Antigua and Barbuda,Americas,ATG
3,Argentina,Americas,ARG
4,Armenia,Asia,ARM
5,Aruba,Americas,ABW
...,...,...,...
155,Uzbekistan,Asia,UZB
156,Venezuela,Americas,VEN
157,Viet Nam,Asia,VNM
158,West Germany,Europe,DEU


In [73]:
gdp_data = pd.merge(gdp_data, countries, on='country_code')

In [74]:
# export gdp data to csv for easier use later 
gdp_data.to_csv('data/gdp_data.csv', index = False)

### Merge import gdp data into data clean

In [75]:
data_clean['year'] = data_clean['Period'].dt.year
data_clean

,Period,Commodity,Province,Country,State,Value,Quantity_kg,Terminated,cpi_val,Value-adjusted,year
0,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",Newfoundland and Labrador,United States,Massachusetts,2454203,42059,None,102.2,3023516.5,2017
1,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",New Brunswick,United States,Massachusetts,20137016,370393,None,102.2,24808298.7,2017
2,2017-05-01,"1605.30.90 - lobster, prepared or preserved, nes",Prince Edward Island,United States,California,31113,1633,None,102.2,38330.4,2017
3,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",Quebec,United States,California,98603,2347,None,102.2,121476.4,2017
4,2017-05-01,"1605.30.90 - lobster, prepared or preserved, nes",Prince Edward Island,United States,New Hampshire,310914,15676,None,102.2,383038.3,2017
...,...,...,...,...,...,...,...,...,...,...,...
56885,1989-02-01,"0306.22.10 - lobsters, (homarus spp), live (te...",British Columbia,Hong Kong,NaN,81553,4538,None,59.7,172082.3,1989
56886,1989-02-01,"0306.12.10 - lobsters in brine, (homarus spp),...",Prince Edward Island,Sweden,NaN,107235,8340,None,59.7,226273.0,1989
56887,1989-02-01,"0306.22.90 - lobsters,(homarus spp), not froze...",Nova Scotia,United States,Massachusetts,16048,1316,None,59.7,33862.4,1989
56888,1989-02-01,"0306.22.10 - lobsters, (homarus spp), live (te...",Nova Scotia,United States,Unknown states,124924,8381,None,59.7,263598.0,1989


In [76]:
data_all = pd.merge(data_clean, gdp_data, left_on  = ['Country', 'year'], right_on = ['Country', 'year'])
data_all

,Period,Commodity,Province,Country,State,Value,Quantity_kg,Terminated,cpi_val,Value-adjusted,year,country_code,GDP-USD,continent
0,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",Newfoundland and Labrador,United States,Massachusetts,2454203,42059,None,102.2,3023516.5,2017,USA,19477336549000.0,Americas
1,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",New Brunswick,United States,Massachusetts,20137016,370393,None,102.2,24808298.7,2017,USA,19477336549000.0,Americas
2,2017-05-01,"1605.30.90 - lobster, prepared or preserved, nes",Prince Edward Island,United States,California,31113,1633,None,102.2,38330.4,2017,USA,19477336549000.0,Americas
3,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",Quebec,United States,California,98603,2347,None,102.2,121476.4,2017,USA,19477336549000.0,Americas
4,2017-05-01,"1605.30.90 - lobster, prepared or preserved, nes",Prince Edward Island,United States,New Hampshire,310914,15676,None,102.2,383038.3,2017,USA,19477336549000.0,Americas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54493,2009-03-01,"0306.22.10 - lobsters, (homarus spp), live (te...",New Brunswick,Mongolia,NaN,11126,674,None,90.2,15519.1,2009,MNG,4583850367.9,Asia
54494,1988-04-01,"0306.22.10 - lobsters, (homarus spp), live (te...",Nova Scotia,"Korea, South",NaN,310,23,None,57.6,676.8,1988,KOR,199591287824.8,Asia
54495,1991-01-01,"0306.22.10 - lobsters, (homarus spp), live (te...",Nova Scotia,Ghana,NaN,2279,176,None,67.1,4275.2,1991,GHA,6603185267.8,Africa
54496,1990-04-01,"0306.12.10 - lobsters in brine, (homarus spp),...",Ontario,Australia,NaN,83950,8050,None,62.5,169191.5,1990,AUS,311420509067.6,Oceania


In [77]:
data_all = data_all.drop(columns = ['country_code', 'Terminated', 'cpi_val', 'Value'])
data_all

,Period,Commodity,Province,Country,State,Quantity_kg,Value-adjusted,year,GDP-USD,continent
0,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",Newfoundland and Labrador,United States,Massachusetts,42059,3023516.5,2017,19477336549000.0,Americas
1,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",New Brunswick,United States,Massachusetts,370393,24808298.7,2017,19477336549000.0,Americas
2,2017-05-01,"1605.30.90 - lobster, prepared or preserved, nes",Prince Edward Island,United States,California,1633,38330.4,2017,19477336549000.0,Americas
3,2017-05-01,"1605.30.10 - lobster, prepared or preserved, f...",Quebec,United States,California,2347,121476.4,2017,19477336549000.0,Americas
4,2017-05-01,"1605.30.90 - lobster, prepared or preserved, nes",Prince Edward Island,United States,New Hampshire,15676,383038.3,2017,19477336549000.0,Americas
...,...,...,...,...,...,...,...,...,...,...
54493,2009-03-01,"0306.22.10 - lobsters, (homarus spp), live (te...",New Brunswick,Mongolia,NaN,674,15519.1,2009,4583850367.9,Asia
54494,1988-04-01,"0306.22.10 - lobsters, (homarus spp), live (te...",Nova Scotia,"Korea, South",NaN,23,676.8,1988,199591287824.8,Asia
54495,1991-01-01,"0306.22.10 - lobsters, (homarus spp), live (te...",Nova Scotia,Ghana,NaN,176,4275.2,1991,6603185267.8,Africa
54496,1990-04-01,"0306.12.10 - lobsters in brine, (homarus spp),...",Ontario,Australia,NaN,8050,169191.5,1990,311420509067.6,Oceania


In [78]:
data_all["Country"] = data_all["Country"].str.strip()

In [79]:
# export clean full data for reuse

data_all.to_csv('data/data_full_clean.csv')

In [80]:
# create commodity list 
commodity_list = pd.DataFrame(data_all['Commodity'].unique())
commodity_list.to_csv('data/commodity-list.csv')

### Monthly US GDP Data


In [81]:
# using country codes, get annual GDP by year for the relevant years in data 

country_code = ['USA']
monthly_gdp = wb.data.DataFrame(gdp_indicator, country_codes, time=range("1988M01", "2023M11")).reset_index()

TypeError: 'str' object cannot be interpreted as an integer

In [82]:
import wbgapi

# Set the World Bank API token (you need to sign up for a token on the World Bank website)
# wbgapi.set_token("YOUR_API_TOKEN")

# Define the indicator code for GDP
indicator_code = "NY.GDP.MKTP.CD"

# Define the country code for the United States
country_code = "USA"

# Set the date range for the monthly data you want
start_date = "2000M01"
end_date = "2023M12"

# Fetch the data
data = wb.get_data(
    indicator=indicator_code,
    country=country_code,
    date=(start_date, end_date),
    mrv=1,  # Monthly data
)

# Print the results
for entry in data:
    print(f"{entry['date']} - GDP: {entry['value']} {entry['unit']['value']}")


AttributeError: module 'wbgapi' has no attribute 'get_data'

# bin

In [ ]:
data_clean.dtypes

In [ ]:
spm = data_clean.loc[data_clean['Country'].str.contains('Miquelon|France|Hong'),['Country', 'Value-adjusted']]
spm.groupby(['Country']).sum()


### 2. EDA on Full Data Set
---
---

In [ ]:
# surpress scientific notation
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
full_data['Province'].value_counts()

In [ ]:
# null values in state are for countries other than US
full_data.info()

In [ ]:
full_data.loc[(full_data['Country'] == 'United States'), 'State'].value_counts()

In [ ]:
exports = data_clean.groupby('Country').mean('Value-adjusted')
exports.sort_values(by='Quantity_kg', ascending=False).head(20)


In [ ]:
# store in a list
top_countries = list(exports.sort_values(by='Quantity_kg', ascending=False).head(20).index)
top_countries

In [ ]:
data_clean.groupby('Country').mean('Quantity_kg').sort_values(by='Quantity_kg', ascending = False)

In [ ]:
pd.DataFrame(data_clean['Country'].value_counts())

In [ ]:
pd.DataFrame(data_clean['Commodity'].value_counts())

In [ ]:
filtered_df = data_clean.loc[data_clean['Country'].isin(top_countries),:]
filtered_df

In [ ]:
filtered_df

In [ ]:
cols = ['Period', 'Country', 'Value($)']
filtered_df = filtered_df[['Period','Commodity', 'Country', 'Quantity', 'Value ($)']]


In [ ]:
data

In [ ]:
sns.scatterplot(x=data['Period'], y=data['Value ($)'], alpha = 0.2)


dont need... got relevant HS codes another way

### Match country names in gdp list to export country list - BIN???

In [ ]:
# filter GDP data by countries in export country list:
gdp_data_filtered = gdp_data[gdp_data['Country Name'].isin(export_country_list)].reset_index(drop=True)
gdp_data_filtered

In [ ]:
# unmatched countries 
unmatched = list(data_clean.loc[~data_clean['Country'].isin(gdp_data['Country Name']), 'Country'].unique())
unmatched = pd.DataFrame(data=unmatched, columns=['unmatched_export_countries'])

In [ ]:
unmatched

In [ ]:
pd.concat([unmatched, gdp_country_list], axis = 1)

In [ ]:
# def country_match(countries_to_match, country_options):
result_df = pd.DataFrame(columns = ['country_to_match','best_match', 'all_matches'] )
for country_index in range(len(unmatched)):
        country_to_match = unmatched.iloc[country_index,0]
        collection = gdp_country_list.iloc[:,0].to_list()
        best_matches = process.extract(country_to_match, collection, scorer=fuzz.ratio)
        new_row = {'country_to_match': country_to_match, 'best_match':best_matches[0][0], 'all_matches':str(best_matches)}
        # print(pd.DataFrame(data = [new_row]))
        result_df = pd.concat([result_df, pd.DataFrame([new_row])], ignore_index=True)

        # result_df = result_df.append(country_to_match, best_matches[0], best_matches, ignore_index = True)
result_df

In [ ]:

bannedWords = ['republic', 'rep', 'of', 'the', ',']

def RemoveBannedWords(toPrint, banned_words):
    # statement = toPrint
    pattern = re.compile(r'\b(?:' + '|'.join(map(re.escape, banned_words)) + r')\b', re.I)
    return pattern.sub("", toPrint)

# # toPrint = unmatched.loc[unmatched['unmatched_export_countries'].str.contains('tanzania'), 'unmatched_export_countries'].items()
# print(type(toPrint))
# print(toPrint)
# print(RemoveBannedWords(toPrint, bannedWords))

In [ ]:
def country_cleaning(df_name, df_column):
    # print(df_name)
    # print(df_name[df_column])
    # df_name.loc[:,df_column] = df_name[df_column].lower()
    df_name[df_column] = df_name[df_column].apply(lambda x: x.lower())
    # df_name[df_column] = df_name[df_column].apply(lambda x: x.replace('of', ''))
    df_name[df_column] = df_name[df_column].apply(lambda x: RemoveBannedWords(x, bannedWords))
    df_name[df_column] = df_name[df_column].apply(lambda x: unidecode(x))
    df_name[df_column] = df_name[df_column].apply(lambda x: x.title())
    return df_name


In [ ]:
country_cleaning(data_clean,'Country')


In [ ]:
# manually update important countries for mapping
gdp_data.loc[gdp_data['Country Name'].str.contains('Korea, Rep.'), 'Country Name'] = 'Korea, South'
gdp_data.loc[gdp_data['Country Name'].str.contains('Hong'), 'Country Name'] = 'Hong Kong'
gdp_data.loc[gdp_data['Country Name'].str.contains('Gambia'), 'Country Name'] = 'Gambia'
gdp_data.loc[gdp_data['Country Name'].str.contains('Egypt'), 'Country Name'] = 'Egypt'
gdp_data.loc[gdp_data['Country Name'].str.contains('Egypt'), 'Country Name'] = 'Macao'
# collapse east /west germany into germany in data clean
data_clean.loc[data_clean['Country'].str.contains('Germany'), 'Country'] = 'Germany'

In [ ]:
# def country_match(countries_to_match, country_options):
result_df = pd.DataFrame(columns = ['country_to_match','best_match', 'all_matches'] )
for country_index in range(len(unmatched)):
        country_to_match = unmatched.iloc[country_index,0].title()
        collection = gdp_country_list.iloc[:,0].to_list()
        best_matches = process.extract(country_to_match, collection, scorer=fuzz.ratio)[0]
        new_row = {'country_to_match': country_to_match, 'best_match':best_matches[0], 'all_matches':str(best_matches)}
        # print(pd.DataFrame(data = [new_row]))
        result_df = pd.concat([result_df, pd.DataFrame([new_row])], ignore_index=True)

        # result_df = result_df.append(country_to_match, best_matches[0], best_matches, ignore_index = True)
result_df

In [ ]:
# QA checks:
# data_clean.loc[data_clean['Country'].str.contains('Gambia')]
# gdp_data.loc[gdp_data['Country Name'].str.contains('Germany')]
# gdp_data.loc[gdp_data['Country Name'].str.contains('Hong')]
gdp_data.loc[gdp_data['Country Name'].str.contains('Macao')]
# data_clean.loc[data_clean['Country'].str.contains('Germany'), 'Country'].unique()

In [ ]:
unmatched['unmatched_export_countries'] = unmatched['unmatched_export_countries'].str.title()
unmatched

In [ ]:
data_clean.loc[~data_clean['Country'].isin(gdp_data['Country Name']), 'Country'].value_counts()

In [ ]:
# unmatched countries again
unmatched = list(data_clean.loc[~data_clean['Country'].isin(gdp_data_filtered['Country Name']), 'Country'].unique())
unmatched = pd.DataFrame(data=unmatched, columns=['unmatched_export_countries'])
unmatched

In [ ]:
df = data_clean.loc[data_clean['Country'].isin(unmatched['unmatched_export_countries']),:]
df[['Country', 'Value']].groupby('Country').sum().sort_values(by='Value', ascending = False)

## Country GDP first attempt

In [ ]:
gdp_data = pd.read_csv('data/gdp-data/gdp_data.csv',engine= 'python', header = 2)
gdp_data

In [ ]:
gdp_data.drop(columns= ['Country Code', 'Indicator Name', 'Indicator Code'], inplace=True)

In [ ]:
gdp_data

In [ ]:
gdp_country_list = pd.DataFrame(data=gdp_data['Country Name'].unique(), columns = ['gdp_country'])
gdp_country_list = gdp_country_list.sort_values(by='gdp_country', ignore_index= True)
gdp_country_list

In [ ]:
hs_8 = pd.read_csv('data/codes_hs8.csv', sep = ',',skiprows = 1, engine='python', skipfooter = 5, index_col=False)
# hs_8 = pd.read_csv('codes_hs8.csv', sep = ',', skipfooter = 5)


In [ ]:
pd.set_option('display.max_colwidth', None)
hs_8['Commodity'] = hs_8['Commodity'].str.lower()

In [ ]:
hs_lobster = hs_8.loc[hs_8['Commodity'].str.contains('lobster'),:]

In [ ]:
hs_lobster['Commodity'].unique()

### Testing univariate ARIMA

With only data from 1 country, for 1 commodity, using Quantity.


In [ ]:
# find country with most complete data accross time. most countries have great data since 2017

In [ ]:
data

In [ ]:
codes = ['0306.22.10', '0306.32.10']



In [ ]:
arima_test = filtered_df[filtered_df['Commodity'].str[:10].isin(codes)].sort_values(by='Period')
arima_test.groupby('Country').count().sort_values(by='Period', ascending = False)
# arima_test

In [ ]:
# filter for one country --- belgium as example
arima_test = arima_test.loc[arima_test['Country'] == 'Belgium']

In [ ]:
# drop cextra columns
arima_test.drop(columns=['Country', 'Value ($)'], inplace = True)
arima_test

In [ ]:
# count records by year, check that there is no overlap
arima_test.groupby(arima_test['Period'].dt.year).count()

In [ ]:
# arima_test = arima_test.drop(columns=['Commodity'])
arima_test = arima_test.reset_index(drop=True)
arima_test

In [ ]:
import seaborn as sns

In [ ]:
sns.lineplot(x = arima_test['Period'], y=arima_test['Quantity'])

In [ ]:
from pandas import datetime
from matplotlib import pyplot
from pandas.plotting import autocorrelation_plot

def parser(x):
 return datetime.strptime('190'+x, '%Y-%m')

series = read_csv('shampoo-sales.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
autocorrelation_plot(series)
pyplot.show()

In [ ]:
japan_data = japan_data[['Period','Quantity']]
japan_data

In [ ]:

japan_data = japan_data.groupby('Period').sum(['Quantity', 'Value ($)']).reset_index(drop=False)
japan_data['Value per kg'] = japan_data['Value ($)'] / japan_data['Quantity']
japan_data

In [ ]:
japan_data['Year'] = japan_data['Period'].dt.year
japan_data



### other

In [ ]:
japan_data_recent = japan_data.loc[japan_data['Period'] > '2020', :].reset_index(drop = True)
japan_data_recent

In [ ]:
sns.lineplot(x=japan_data_recent['Year'], y=japan_data_recent['Quantity'])

In [ ]:
data

In [ ]:
sns.lineplot(x=data['Period'], y=data['Value ($)'])